# Task 2

## Original exercise number

Exercise 14-4

## Description

In a large collection of MP3 files, there may be more than one copy of the same song, stored in different directories or with different file names. The goal of this exercise is to search for duplicates.

1. Write a program that searches a directory and all of its subdirectories, recursively, and returns an array of complete paths for all files with a given suffix (like .mp3).

2. To recognize duplicates, you can use `md5sum` or `md5` to compute a “checksum” for each files. If two files have the same checksum, they probably have the same contents.

3. To double-check, you can use the Unix command `diff`.

## Solution

NO GUARANTEE THAT THE SOLUTION WILL WORK OR WORKS CORRECTLY! USE IT AT
YOUR OWN RISK!

### Functions

In [ ]:
# I will use (and modify) walk function from this chapter
# ".txt" and "txt" are two different suffixes
function walk_paths_get_file_names_with_suffix(
        start_dirname::String, suffix::String,
        accumulator::Vector{String}=Vector{String}())::Vector{String}
    for name in readdir(start_dirname)
        path::String = joinpath(start_dirname, name)
        if isdir(path)
            walk_paths_get_file_names_with_suffix(path, suffix, accumulator)
        elseif isfile(path) && endswith(path, suffix)
            push!(accumulator, path)
        end
    end
    return accumulator
end

In [ ]:
# gets checksum for a file (the file must exist, it is not verified)
function get_md5sum(file::String)::String
    return read(`md5sum $file`, String) |> x -> split(x)[1]
end

In [ ]:
function get_diff(file1::String, file2::String)::String
    return read(`diff -q $file1 $file2`, String)
end

In [ ]:
function are_strings_equal(str1::String, str2::String)::Bool
    return strip(str1) == strip(str2)
end

In [ ]:
function are_files_duplicates(file1::String, file2::String)::Bool
    return (are_strings_equal(get_md5sum(file1), get_md5sum(file2))
            &&
            strip(get_diff(file1, file2)) == "")
end

In [ ]:
# walks paths recursively, list duplicate files with a given suffix
# ".txt" and "txt" are two different suffixes
function walk_paths_list_duplicates(start_dirname::String, suffix::String)
    files::Vector{String} = walk_paths_get_file_names_with_suffix(start_dirname, suffix)
    duplicates::Vector{Tuple{String, String}} = []
    for i in eachindex(files), j in eachindex(files)
        if (j > i) && are_files_duplicates(files[i], files[j])
            push!(duplicates, (files[i], files[j]))
        end
    end
    return duplicates
end

## Testing

In [ ]:
walk_paths_list_duplicates("./", ".txt")